Open questions:
Why he used second column of model predict_proba? Binary classification??


In [18]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score, f1_score, precision_score, recall_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd


from lime import lime_tabular
import sys

In [19]:
# Load the Breast Cancer dataset
data = load_breast_cancer()

# Create a DataFrame with feature names
X = pd.DataFrame(data.data, columns=data.feature_names)

# Create a Series for the target variable
y = pd.Series(data.target, name='target')


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [20]:
# Initialize and train the XGBoost Classifier
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [21]:
y_pred = model.predict(X_test)

In [22]:
y_pred

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96        63
           1       0.98      0.97      0.98       108

    accuracy                           0.97       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171



## LIME

In [24]:
#!pip install lime

In [25]:
number_of_test_points: int = len(X_test)
DIMENSION = X_test.shape[1]

coefficients_shape = (number_of_test_points, DIMENSION + 1 ) # +1 for the intercept
lime_coefficients: np.ndarray = np.zeros(shape=coefficients_shape, dtype=float)
lime_pred_estimation: np.ndarray = np.zeros(shape=number_of_test_points, dtype=float)
ground_truth_coefficients: np.ndarray = np.zeros(shape=coefficients_shape, dtype=float)

lime_explainer = lime_tabular.LimeTabularExplainer(training_data=X_train.to_numpy(), mode="classification")
def get_local_explanation_lime(points: pd.DataFrame, explainer, predict_function, coefficients, pred_estimation, dim):

    for i, instance in points.reset_index(drop=True).iterrows():
        exp = explainer.explain_instance(instance, predict_fn=predict_function, num_features=dim)
        pred_estimation[i] = exp.local_pred[0]

        # progress bar
        sys.stdout.write('\r')
        progress = i/points.shape[0]
        sys.stdout.write("[%-100s] %d%%" % ('='*int(progress*100), progress*100))
        sys.stdout.write(" -> " + str(round(progress, 4)))
        sys.stdout.flush()

    return coefficients, pred_estimation


lime_coefficients, lime_pred_estimation = get_local_explanation_lime(
    X_test,
    lime_explainer,
    model.predict_proba,
    lime_coefficients,
    lime_pred_estimation,
    DIMENSION)

[=================================================================================================== ] 99% -> 0.9942

In [26]:
lime_pred_estimation = np.clip(lime_pred_estimation, 0, 1)
lime_pred_estimation

array([0.91241203, 0.        , 0.        , 1.        , 1.        ,
       0.        , 0.        , 0.18387147, 0.57916265, 1.        ,
       0.95132111, 0.00594712, 1.        , 0.26358446, 1.        ,
       0.        , 1.        , 1.        , 1.        , 0.        ,
       0.90250064, 1.        , 0.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.44230239, 1.        , 0.        , 0.95078043,
       1.        , 0.29211052, 1.        , 1.        , 0.93456866,
       0.85345916, 1.        , 1.        , 0.74799106, 1.        ,
       0.        , 0.        , 0.72498147, 1.        , 1.        ,
       1.        , 1.        , 0.        , 0.29166867, 1.        ,
       1.        , 0.        , 0.        , 0.77852816, 1.        ,
       0.80826683, 0.        , 0.        , 1.        , 1.        ,
       0.        , 0.        , 1.        , 0.06969257, 1.     

In [27]:
model_pred = model.predict_proba(X_test)[:,1]
model_pred

array([9.9341184e-01, 2.2777125e-04, 2.9362887e-04, 9.9955302e-01,
       9.9987042e-01, 4.6230009e-04, 6.2651420e-04, 5.7528716e-02,
       3.2596284e-01, 9.9953854e-01, 9.7041851e-01, 3.1744672e-03,
       9.9686426e-01, 1.2310396e-02, 9.9882644e-01, 8.2263915e-04,
       9.9907994e-01, 9.9988723e-01, 9.9927372e-01, 5.6035578e-04,
       9.9157214e-01, 9.9823940e-01, 5.7713460e-04, 9.9747038e-01,
       9.9930716e-01, 9.9943358e-01, 9.9975175e-01, 9.9074179e-01,
       9.9946696e-01, 4.4600532e-04, 9.9954069e-01, 9.9933213e-01,
       9.7257847e-01, 9.9902785e-01, 9.9945432e-01, 9.9741817e-01,
       3.0017245e-01, 9.9693918e-01, 1.9351627e-04, 9.9889272e-01,
       9.9983907e-01, 1.3408650e-03, 9.9928695e-01, 9.9964321e-01,
       9.6204013e-01, 9.9107307e-01, 9.9860793e-01, 9.8314583e-01,
       9.8331082e-01, 9.9946374e-01, 2.1170442e-04, 7.2574330e-04,
       9.5647967e-01, 9.6435249e-01, 9.9981648e-01, 9.9946386e-01,
       9.9977630e-01, 4.1851730e-04, 8.4789749e-03, 9.9942052e

In [28]:
predict_function = model.predict
model_pred_class = predict_function(X_test).values \
    if type(predict_function(X_test)) in [pd.DataFrame, pd.Series] \
    else predict_function(X_test)

model_pred_class

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [29]:
y_pred_class_lime = np.where(lime_pred_estimation> 0.5, 1, 0)
y_pred_class_lime

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [30]:
import EBM

In [31]:
ebm = EBM.EBM(model, X_train, y_train, mode='classification')

In [32]:
ebm_pred_estimation = ebm.predict_proba(X_test)[:, 1]
ebm_pred_estimation

array([9.83973333e-01, 3.83149669e-04, 4.73564289e-03, 9.94338146e-01,
       9.99636157e-01, 4.56180378e-05, 3.64536187e-04, 2.61223970e-02,
       1.68606145e-01, 9.98635804e-01, 9.76674897e-01, 2.77530899e-03,
       9.92657373e-01, 2.09285465e-02, 9.99832853e-01, 9.40776413e-04,
       9.94660665e-01, 9.99587542e-01, 9.99656709e-01, 6.26865554e-04,
       9.59400106e-01, 9.97770953e-01, 2.92350166e-05, 9.99319329e-01,
       9.98031357e-01, 9.96591097e-01, 9.98538396e-01, 9.94510219e-01,
       9.98328218e-01, 1.14871106e-04, 9.98515013e-01, 9.99537476e-01,
       9.91215997e-01, 9.95350156e-01, 9.99214658e-01, 9.98724649e-01,
       3.49708563e-01, 9.98439145e-01, 1.84016346e-03, 9.62965699e-01,
       9.98934732e-01, 5.12182873e-04, 9.98372648e-01, 9.99224712e-01,
       9.76561976e-01, 9.97414770e-01, 9.96096204e-01, 9.96649469e-01,
       9.94272407e-01, 9.98526345e-01, 1.51431371e-03, 2.73536301e-05,
       7.17748465e-01, 9.88915716e-01, 9.99427327e-01, 9.98472868e-01,
      

In [33]:
y_pred_class_ebm = ebm.predict(X_test)
y_pred_class_ebm

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1])

In [34]:
import MimicExpl

In [35]:
mimic = MimicExpl.MimicExpl(model, X_train, y_train, mode='classification')

[LightGBM] [Info] Number of positive: 249, number of negative: 149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000396 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3978
[LightGBM] [Info] Number of data points in the train set: 398, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.625628 -> initscore=0.513507
[LightGBM] [Info] Start training from score 0.513507
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [36]:
y_pred_class_mimic = mimic.predict(X_test)
y_pred_class_mimic

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [37]:
mimic_pred_estimation = mimic.predict_proba(X_test)
mimic_pred_estimation

array([9.99240536e-01, 4.57556338e-05, 1.93930096e-04, 9.99942703e-01,
       9.99983154e-01, 3.83948794e-05, 4.65589388e-05, 7.81087471e-02,
       3.33375976e-02, 9.99911767e-01, 9.97476555e-01, 1.81698989e-04,
       9.99604027e-01, 3.54302231e-02, 9.99964049e-01, 3.95444304e-05,
       9.99938826e-01, 9.99958226e-01, 9.99969117e-01, 5.77702257e-05,
       9.93931590e-01, 9.99921695e-01, 3.03335996e-05, 9.99759848e-01,
       9.99893324e-01, 9.99873866e-01, 9.99967731e-01, 9.99516420e-01,
       9.99949549e-01, 2.88485394e-05, 9.99953184e-01, 9.99967106e-01,
       9.95497783e-01, 9.99911679e-01, 9.99970894e-01, 9.99876469e-01,
       7.19781539e-01, 9.99885204e-01, 4.54406081e-05, 9.99378184e-01,
       9.99981547e-01, 3.36774003e-04, 9.99956124e-01, 9.99974828e-01,
       9.54059626e-01, 9.98724019e-01, 9.99904287e-01, 9.99538599e-01,
       9.97638282e-01, 9.99968298e-01, 5.54188333e-05, 2.67265081e-05,
       9.52036524e-01, 9.96988488e-01, 9.99975025e-01, 9.99807442e-01,
      

In [38]:
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, precision_score, recall_score
import pandas as pd

def create_metrics_dataframe(estimator_names, pred_estimations, y_pred_classes, model_pred, model_pred_class):
    """
    Creates a DataFrame with evaluation metrics for given estimators.

    Parameters:
    estimator_names (list): List of estimator names.
    pred_estimations (list): List of predicted estimations for each estimator.
    y_pred_classes (list): List of predicted classes for each estimator.
    model_pred (array-like): Model's predicted values.
    model_pred_class (array-like): Model's predicted classes.

    Returns:
    pd.DataFrame: DataFrame with evaluation metrics.
    """
    metrics = {
        'MSE': [mean_squared_error(pred, model_pred) for pred in pred_estimations],
        'ACCURACY': [accuracy_score(y_pred, model_pred_class) for y_pred in y_pred_classes],
        'F1_SCORE': [f1_score(y_pred, model_pred_class) for y_pred in y_pred_classes],
        'PRECISION': [precision_score(y_pred, model_pred_class) for y_pred in y_pred_classes],
        'RECALL': [recall_score(y_pred, model_pred_class) for y_pred in y_pred_classes]
    }

    results = pd.DataFrame(metrics, index=estimator_names).T
    return results



In [39]:
estimator_names = ['lime', 'ebm', 'mimic']
pred_estimations = [lime_pred_estimation, ebm_pred_estimation, mimic_pred_estimation]
y_pred_classes = [y_pred_class_lime, y_pred_class_ebm, y_pred_class_mimic]

results = create_metrics_dataframe(estimator_names, pred_estimations, y_pred_classes, model_pred, model_pred_class)
results

,lime,ebm,mimic
MSE,0.012413,0.009377,0.006083
ACCURACY,0.959064,0.970760,0.976608
F1_SCORE,0.967742,0.976744,0.981308
PRECISION,0.981308,0.981308,0.981308
RECALL,0.954545,0.972222,0.981308
